<a href="https://colab.research.google.com/github/adithi20/Data-structures/blob/main/Book_Recommendation_Engine_using_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy pandas scikit-learn


In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
!unzip book-crossings.zip


--2023-11-17 21:05:35--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   164MB/s    in 0.2s    

2023-11-17 21:05:36 (164 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [5]:
# import csv data into dataframes
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'
df_books = pd.read_csv(
    books_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [6]:
# Section 1: Data Cleaning and Filtering

# Filter users with less than 200 ratings and books with less than 100 ratings
user_ratings_count = df_ratings['user'].value_counts()
book_ratings_count = df_ratings['isbn'].value_counts()

filtered_users = user_ratings_count[user_ratings_count >= 200].index
filtered_books = book_ratings_count[book_ratings_count >= 100].index

df_filtered_ratings = df_ratings[df_ratings['user'].isin(filtered_users) & df_ratings['isbn'].isin(filtered_books)]

In [8]:
# Section 2: Create Pivot Table for Nearest Neighbors

# Create a pivot table with users as rows, books as columns, and ratings as values
pivot_table = df_filtered_ratings.pivot(index='user', columns='isbn', values='rating').fillna(0)
# Convert the pivot table to a sparse matrix
matrix = csr_matrix(pivot_table.values)

In [9]:
# Section 3: Build and Fit Nearest Neighbors Model

# Create a Nearest Neighbors model with cosine similarity
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)

# Fit the model to the data
model_knn.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [25]:
# Section 4: Define the Recommendation Function

# Function to return recommended books
def get_recommends(book=""):
    # Find the index of the book in the pivot table
    book_index = pivot_table.columns.get_loc(book)

    # Get the distances and indices of the nearest neighbors
    distances, indices = model_knn.kneighbors(matrix[:, book_index].reshape(1, -1), n_neighbors=6)

    # Create a list of recommended books with distances
    books = get_recommends("Classical Mythology")

    recommended_books = [book, []]

    for i in range(1, len(distances.flatten())):
        recommended_books[1].append([pivot_table.columns[indices.flatten()[i]], distances.flatten()[i]])

    return recommended_books

In [26]:
print(df_books['title'])


0                                       Classical Mythology
1                                              Clara Callan
2                                      Decision in Normandy
3         Flu: The Story of the Great Influenza Pandemic...
4                                    The Mummies of Urumchi
                                ...                        
271374                           There's a Bat in Bunk Five
271375                              From One to One Hundred
271376    Lily Dale : The True Story of the Town that Ta...
271377                          Republic (World's Classics)
271378    A Guided Tour of Rene Descartes' Meditations o...
Name: title, Length: 271379, dtype: object


In [27]:
# Cell 3: Import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

# Clean book titles by removing leading and trailing whitespaces
df_books['title'] = df_books['title'].str.strip()


In [28]:
# Check if the book title exists in the dataset
df_books[df_books['title'] == 'Classical Mythology']



,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
95235,0801319536,Classical Mythology,Mark P. O. Morford


In [31]:
# Section 5: Testing
def test_book_recommendation():
    test_pass = True
    try:
        recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    except KeyError as e:
        print(f"KeyError: {e}")
        print("Please make sure the book title is present in the dataset.")
        return

    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False
    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
    for i in range(4):
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

# Run the test
test_book_recommendation()


KeyError: "Where the Heart Is (Oprah's Book Club (Paperback))"
Please make sure the book title is present in the dataset.


In [32]:
import pandas as pd

# Assuming df_ratings is the DataFrame containing ratings data
# Add this before the testing cell
df_ratings_visualization = df_ratings.merge(df_books, on='isbn')[['title', 'user', 'rating']]

# Display the first few rows of the DataFrame
print(df_ratings_visualization.head())


                  title    user  rating
0  Flesh Tones: A Novel  276725     0.0
1  Flesh Tones: A Novel    2313     5.0
2  Flesh Tones: A Novel    6543     0.0
3  Flesh Tones: A Novel    8680     5.0
4  Flesh Tones: A Novel   10314     9.0


In [33]:
import pandas as pd

# Assuming df_ratings is the DataFrame containing ratings data
# Add this before the testing cell
df_ratings_visualization = df_ratings.merge(df_books, on='isbn')[['title', 'user', 'rating']]

# Display the data as a table
display(df_ratings_visualization)


,title,user,rating
0,Flesh Tones: A Novel,276725,0.0
1,Flesh Tones: A Novel,2313,5.0
2,Flesh Tones: A Novel,6543,0.0
3,Flesh Tones: A Novel,8680,5.0
4,Flesh Tones: A Novel,10314,9.0
...,...,...,...
1031170,Mostly Harmless,276688,0.0
1031171,Gray Matter,276688,7.0
1031172,Triplet Trouble and the Class Trip (Triplet Tr...,276690,0.0
1031173,A Desert of Pure Feeling (Vintage Contemporaries),276704,0.0


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['user'].plot(kind='hist', bins=20, title='user')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['rating'].plot(kind='hist', bins=20, title='rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='index', y='user', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['rating']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_6.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('rating')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_8['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['user'].plot(kind='line', figsize=(8, 4), title='user')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['rating'].plot(kind='line', figsize=(8, 4), title='rating')
plt.gca().spines[['top', 'right']].set_visible(False)